# Bank Statement Analysis 

Jupyter Notebook created to categorize .csv based bank statment expances.  

Currently suppoorted bank account export format is `ING Bank Śląski` which can aggreagate expances form other banks.

# Export data from Bank accounts


**ING Bank Śląski** -  Export the accounts you would like to do analysis for. Usually I'm esporting expances from all accounts except `Rachunki` bank account to see how much money was spenf to things like grocery. Exported file have `.csv` format and `cp-1250` encoding.

Data presnetation article
https://www.shanelynn.ie/data-visualisation-in-python-pycon-dublin-2018-presentation/




## Input Data

In [3]:
# ING_CSV = "data/2024.csv"
ING_CSV = "data/ING_MILL_2025.csv"

## Categorization

In [18]:
CATEGORY = {}

# 1. ING Bank Śląski



## 1.1 Import Data from ING Bank Sląski

Import raw data from .csv file.

`header` - is an index of non empty row in input csv file. Will be used as `df` column name.


In [4]:
import pandas as pd
from tabulate import tabulate

from helper import get_ing_header_row_index

# CSV file settings 
ING_ENCODING = "cp1250"
ING_SEP = ";"

ing_raw_df = pd.read_csv(
    ING_CSV,
    encoding=ING_ENCODING,
    sep=ING_SEP,
    header=get_ing_header_row_index(
        ING_CSV, ING_ENCODING
    ),
)

# Trim data frame - Drop last row which is empty
ing_raw_df.drop(ing_raw_df.tail(1).index, inplace=True)

## 1.2 - Data preparation

Ceanup the data from ING Bank account to extract only usefull columns

* Trim data fame to get only usefull columns for analysis
* Manipulate data
* Set data types


In [8]:
ing_raw_df.head(10)

,Data transakcji,Data księgowania,Dane kontrahenta,Tytuł,Nr rachunku,Nazwa banku,Szczegóły,Nr transakcji,Kwota transakcji (waluta rachunku),Waluta,...,Waluta.1,Kwota płatności w walucie,Waluta.2,Konto,Bank,Saldo po transakcji,Waluta.3,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,2025-04-19,NaN,UL. KOSCIUSZKI 44E GORLICE POL,Wypłata gotówki 19.04.2025 Nr karty 4246xx399...,NaN,NaN,NaN,NaN,NaN,NaN,...,PLN,NaN,NaN,Wspólne,ING Bank Śląski S.A.,"0,20",PLN,NaN,NaN,NaN
1,2025-04-19,2025-04-19,SODEL ALBERT ŁUKASZ SADOWA 28 55-002 DOBRZYKO...,Przelew własny,'82105014321000009090903783 ',ING Bank Śląski S.A.,PRZELEW,'202510997201639242',"1,00",PLN,...,NaN,NaN,NaN,Wspólne,ING Bank Śląski S.A.,"500,20",PLN,NaN,NaN,NaN
2,2025-04-19,2025-04-19,SODEL ALBERT ŁUKASZ SADOWA 28 55-002 DOBRZYKO...,Przelew własny,'40105014321000009720965129 ',ING Bank Śląski S.A.,PRZELEW,'202510997201639242',"-1,00",PLN,...,NaN,NaN,NaN,Osobiste,ING Bank Śląski S.A.,"8,31",PLN,NaN,NaN,NaN
3,2025-04-19,2025-04-19,SODEL ALBERT ŁUKASZ SADOWA 28 DOBRZYKOWICE,Przelew własny,'76105014321000009731987658 ',ING Bank Śląski S.A.,PRZELEW,'202510997201639000',"310,00",PLN,...,NaN,NaN,NaN,Wspólne,ING Bank Śląski S.A.,"499,20",PLN,NaN,NaN,NaN
4,2025-04-19,NaN,APPLE.COM/BILL APPLE.COM/BIL IRL,Płatność kartą 19.04.2025 Nr karty 4246xx7661,NaN,NaN,NaN,NaN,NaN,NaN,...,PLN,NaN,NaN,Rachunki,ING Bank Śląski S.A.,"3246,21",PLN,NaN,NaN,NaN
5,2025-04-19,NaN,APPLE.COM/BILL APPLE.COM/BIL IRL,Płatność kartą 19.04.2025 Nr karty 4246xx7661,NaN,NaN,NaN,NaN,NaN,NaN,...,PLN,NaN,NaN,Rachunki,ING Bank Śląski S.A.,"3308,20",PLN,NaN,NaN,NaN
6,2025-04-19,NaN,McDonalds Mogilany POL,Płatność kartą 19.04.2025 Nr karty 4246xx0146,NaN,NaN,NaN,NaN,NaN,NaN,...,PLN,NaN,NaN,Osobiste,ING Bank Śląski S.A.,"9,31",PLN,NaN,NaN,NaN
7,2025-04-19,NaN,Autopay Mobility Warszawa POL,Płatność kartą 19.04.2025 Nr karty 4246xx3990,NaN,NaN,NaN,NaN,NaN,NaN,...,PLN,NaN,NaN,Wspólne,ING Bank Śląski S.A.,"189,20",PLN,NaN,NaN,NaN
8,2025-04-19,NaN,Autopay Mobility Warszawa POL,Płatność kartą 19.04.2025 Nr karty 4246xx3990,NaN,NaN,NaN,NaN,NaN,NaN,...,PLN,NaN,NaN,Wspólne,ING Bank Śląski S.A.,"206,20",PLN,NaN,NaN,NaN
9,2025-04-19,NaN,ORLEN STACJA NR 459 LANY POL,Płatność kartą 19.04.2025 Nr karty 4246xx3990,NaN,NaN,NaN,NaN,NaN,NaN,...,PLN,NaN,NaN,Wspólne,ING Bank Śląski S.A.,"223,20",PLN,NaN,NaN,NaN


In [ ]:
# *** Trim data frame ***

"""
Fetch from raw data  frame only usefull columns, nothing more than that.
If needed first copy the data around to have coherent data set
"""



# *** Set Column Data types ***
"""
Might be usefull to analyse data afterwards.
"""

# ------------------------------------------------------------------------

# Set Data Types for columns
float_columns = [
    "Kwota transakcji (waluta rachunku)",
    "Kwota blokady/zwolnienie blokady",
    "Saldo po transakcji",
]

for col in float_columns:
    ing_raw_df[col] = ing_raw_df[col].str.replace(",", ".").astype(float)

# *** Manipulate data ***

#TODO: Why this is done like this?
# Set coherent column meaning for Millennium and ING by copying data from 'Tytuł' to 'Dane kontrahenta'
ing_raw_df["Dane kontrahenta"] = ing_raw_df.apply(
    lambda row: (
        row["Tytuł"]
        if row["Dane kontrahenta"] == "  "
        else row["Dane kontrahenta"]
    ),
    axis=1,
)
# from helper import print_df
# print_df(ing_raw_df)
# ing_raw_df

# TODO: Evalueate if we need to fill the missing values in the 'Data transakcji' column

# # If Data transakcji is  empty, fill it with Data księgowania
# df.loc[
#     df["Kwota transakcji (waluta rachunku)"].isna(),
#     "Kwota transakcji (waluta rachunku)",
# ] = df["Kwota blokady/zwolnienie blokady"]

# # For all blocked transactions fill the Kwota transakcji with tha blocked amount.
# df.loc[df["Data transakcji"].isna(), "Data transakcji"] = df["Data księgowania"]


## 1.3 ING Data analysis



In [19]:
pass